In [2]:
 import json
import requests
import string
from collections import Counter
import time


ACCESSTOKEN  = 'EAACEdEose0cBAF4jtckRxbok2QKqmOR0BEA5N11neqxnLeBLId5Qgrlj7n0fXYhxaOWv8yjcIXhI9OFbR5WnpHhZBQZBQoFUOcHvA2f6vWnwYRbRr53JqptvXEKiijq52fHueym62aCBgHC31rnfPM3WN2oOyPVsJpYiJRYgZDZD'
PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?limit=100&fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)

jd = json.loads(res.text)

PostID= jd['data'][0]['id']
print jd['data'][0]['shares']['count']
len(jd['data'])




31


100

In [3]:
#粉絲抓取

PostUrl ='https://graph.facebook.com/v2.5/15925638948?fields=likes&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)

jd = json.loads(res.text)

Fans= jd['likes']

print Fans
def inserFans():
    command="INSERT INTO FBFans (Crawler_Time,Fans) VALUES (?,?)"

    cursor.execute(command,(Crawler_Time,Fans))
    conn.commit()


6636560


In [4]:

import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()

In [5]:
def SharesSecond(PostNO):

    cursor.execute("select * from FBPostChange where PostNO=? order by LikeCount desc",PostNO)
    Postrows = cursor.fetchall()
    
    try:
        Postrows[0]
        return Postrows
    except:
        return [(0,0,0,0,0)]
SharesSecond(1)

[(1, 233, 11, 31, u'11', 233, 11, 31)]

In [6]:
#判斷起始NO
#IDrow

cursor.execute("select * from FBPostData   order by PostNO asc")

def Second():
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
    cursor = conn.cursor()
    cursor.execute("select * from FBPostData   order by PostNO asc")
    global rows
    rows = cursor.fetchall()
    global IDrows
    
    IDrows=[]
#自動判斷1 或是第二次以上NO   
    try:
        PostNO= rows[-1][0]+1
        for row in rows:
            IDrows.append( row[-1])
        return PostNO
    except:
        return 1
Second()

3

In [7]:
#偵測PO文NO

def PostNOdetect(id):
    
    
    for row in rows:
        if row[-1]==id:
            NO=row[0]
            
    return NO

#PostNOdetect('15925638948_10154486947508949')

In [8]:
#PO文 抓入SQL

def insertSQL(id,PostNO):
    status='new'
    
    #如果是舊的PO文
    if(id in IDrows ):
        status='old'
        PostNO=PostNOdetect(id)
    
    if(status=='new'):
        command="INSERT INTO FBPostData (PostNO,Created_Time,Message,ID,AttatchTitle,AttatchMessage,Crawler_Time) VALUES (?,?,?,?,?,?,?)"

        cursor.execute(command,(PostNO,created_time,Postmessage,id,attatchTitle,attatchMessage,Crawler_Time))
        
    command="INSERT INTO FBPostChange (PostNO,LikeCount,CommentCount,ShareCount,Crawler_Time,LikeIncrease,CommentIncrease,ShareIncrease) VALUES (?,?,?,?,?,?,?,?)"

    cursor.execute(command,(PostNO,LikeCount,CommentCount,ShareCount,Crawler_Time,LikeIncrease,CommentIncrease,ShareIncrease))
    
# rows = cursor.fetchall()
# for row in rows:

In [9]:
#判斷起始NO



def CommentSecond(PostNO):
    global CommentIDrows
    global Commentrows
    CommentIDrows=[]
    Commentrows=[]
    
    cursor.execute("select * from FBCommentData where PostNO=?  order by CommentNO asc",PostNO)
    Commentrows = cursor.fetchall()
    try:
        CommentNO= Commentrows[-1][1]+1
        for row in Commentrows:
            CommentIDrows.append( row[-3])

        return CommentNO
    except:
        return 1
CommentSecond(3)


1

In [10]:
def CommentNOdetect(id):
    for row in Commentrows:
        if row[-3]==id:
            CommentNO=row[1]
    return CommentNO

#CommentNOdetect('10154486947508949_10154487300663949')

In [11]:
def InsertComment(CommentID,CommentNO):
    CommentStatus='new'
    if(CommentID in CommentIDrows ):
        CommentStatus='old'
        CommentNO2=CommentNOdetect(CommentID)
        command="INSERT INTO FBCommentChange (PostNO,CommentNO,LikeCount,ReplyCount,Crawler_Time) VALUES (?,?,?,?,?)"
        cursor.execute(command,(PostNO,CommentNO2,like_count,comment_count,Crawler_Time))
        return CommentNO
    
    if(CommentStatus=='new'):
        command="INSERT INTO FBCommentData (PostNO,CommentNO,Name,Message,CommentID,UserID,Crawler_Time) VALUES (?,?,?,?,?,?,?)"
        cursor.execute(command,(PostNO,CommentNO,Name,message,CommentID,UserID,Crawler_Time))
        command="INSERT INTO FBCommentChange (PostNO,CommentNO,LikeCount,ReplyCount,Crawler_Time) VALUES (?,?,?,?,?)"
        cursor.execute(command,(PostNO,CommentNO,like_count,comment_count,Crawler_Time))
        return CommentNO+1


In [12]:
def Comment(PostID):
    start =time.time()
    
    CommentUrl ='https://graph.facebook.com/v2.5/'+PostID+'/comments?fields=like_count,message,comment_count,from,id&limit=200&access_token='+ACCESSTOKEN

    CommentRes = requests.get(CommentUrl)

    CommentJd = json.loads(CommentRes.text)
    
    
    global like_count
    global message
    global comment_count
    global Name
    global UserID
    global CommentID
    
    global CommentStatus
    
    CommentStatus='new'
    CommentNO=CommentSecond(PostNO)
    
    
    
    try:
        bug=CommentJd['paging']
    except:
        print PostID
        return 0
    
    
    end =time.time()
    time1= end-start
    HowMuch=str(len(CommentJd['data']))
    print '評論載入成功:'+HowMuch+'筆 '+str(time1)+'秒'
    
    
   
    
    start =time.time()
    
    while 'next' in CommentJd['paging'] :
        
        
        for data in CommentJd['data']:
            like_count= data['like_count']
            message=data['message']
            comment_count=data['comment_count']

            Name= data['from']['name']
            UserID= data['from']['id']
            CommentID=data['id']
            
                
            #插入    
            CommentNO=InsertComment(CommentID,CommentNO)
            
            if(CommentID==CommentJd['data'][-1]['id']):

                CommentUrl=CommentJd['paging']['next']
                time.sleep(1)
                CommentRes=requests.get(CommentUrl)
                CommentJd = json.loads(CommentRes.text)

            
            
            
    for data in CommentJd['data']:
        like_count= data['like_count']
        message=data['message']
        comment_count=data['comment_count']

        Name= data['from']['name']
        UserID= data['from']['id']
        CommentID=data['id']
        #插入
        CommentNO=InsertComment(CommentID,CommentNO)
        
    end=time.time()
    time1=end-start
    print '評論輸入完畢:'+str(CommentNO-1)+"筆 "+str(time1)+'秒'
    
        
    return CommentNO-1
    
    
    
# Comment('15925638948_10154512960348949')    
    
    

In [13]:
#判斷起始NO



def LikeSecond(PostNO):
    global LikeIDrows
    global Likerows
    LikeIDrows=[]
    Likerows=[]
    
    cursor.execute("select * from FBPostLikeChange where PostNO=?  order by LikeNO asc",PostNO)
    Likerows = cursor.fetchall()
    try:
        LikeNO= Likerows[-1][1]+1
        for row in Likerows:
            LikeIDrows.append( row[2])

        return LikeNO
    except:
        return 1
LikeSecond(2)


1

In [163]:
def LikeNOdetect(id):
    for row in Likerows:
        if row[2]==id:
            LikeNO=row[1]
    return LikeNO

#CommentNOdetect('10154486947508949_10154487300663949')

In [164]:
def InsertLike(UserID,LikeNO):
    
    LikeStatus='new'
    if(UserID in LikeIDrows ):
        LikeStatus='old'
        
        return LikeNO
    
    if LikeStatus=='new':
        
#         command="INSERT INTO FBPostLikeChange (PostNO,LikeNO,UserID,UserName,Crawler_Time) VALUES (?,?,?,?,?)"

#         cursor.execute(command,(PostNO,LikeNO,UserID,UserName,Crawler_Time))
        return LikeNO+1

In [165]:
def Like(PostID):
    
    global UserName
    global UserID
    global CommentStatus
   
    start=time.time()
    CommentStatus='new'
    
    LikeNO= LikeSecond(PostNO)
    
    LikeUrl ='https://graph.facebook.com/v2.5/'+PostID+'/likes?pretty=0&limit=1000&access_token='+ACCESSTOKEN

    LikeRes = requests.get(LikeUrl)

    LikeJd = json.loads(LikeRes.text)
    end =time.time()
    time1= end-start
    HowMuch=str(len(LikeJd['data']))
    print '讚數載入成功:'+HowMuch+'個 '+str(time1)+'秒'
    
    start=time.time()
    try:
        while 'next' in LikeJd['paging']:
            for data in LikeJd['data']:
                UserName= data['name']
                UserID= data['id']
                LikeNO=InsertLike(UserID,LikeNO)
                if(UserID== LikeJd['data'][-1]['id']):
                    LikeUrl=LikeJd['paging']['next']
                    #time.sleep(0.1)
                    LikeRes=requests.get(LikeUrl)
                    LikeJd = json.loads(LikeRes.text)
                
    except:
        return 0
            
        
    for data in LikeJd['data']:
        UserName= data['name']
        UserID= data['id']
        LikeNO=InsertLike(UserID,LikeNO)
        
        
    end =time.time()
    time1= end-start
    HowMuch=str(len(LikeJd['data']))
    print '讚數輸入成功:'+str(LikeNO-1)+'個 '+str(time1)+'秒'
    return LikeNO-1
    
    

In [166]:
#開爬

import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.68.48;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()


#狀態初始化
end=0
status='new'
IDrows=[]


PostNO=Second()


#粉絲

PostUrl ='https://graph.facebook.com/v2.5/15925638948?fields=likes&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)
Fans= jd['likes']


#PO文
PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?limit=100&fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)




#時間
Crawler_Time = raw_input("Crawler_Time:")
print Crawler_Time

# inserFans()



while( 'next' in jd['paging'] ):
    status='new'
    if end==1:
        break
    
    for post in jd['data']:
        attatchTitle=""
        attatchMessage=""
        CommentCount=0
        LikeCount=0
        ShareCount=0
        Postmessage=""
        
        #例外
        if('message'in post):
            Postmessage=post['message']
        if('shares'in post):
            ShareCount=post['shares']['count']
        created_time=post['created_time']
            
        id=post['id']

        
        #attachments
        if('attachments' in post):
            if('title' in post['attachments']['data'][0]):
                attatchTitle=post['attachments']['data'][0]['title']
            if('description' in post['attachments']['data'][0]):
                attatchMessage=post['attachments']['data'][0]['description']

        if(id in IDrows ):
            status='old'
            PostNO=PostNOdetect(id)
        
        Postrow=SharesSecond(PostNO)
       
        PreLikeCount=Postrow[0][1]
        PreCommentCount=Postrow[0][2]
        PreShareCount=Postrow[0][3]

        CommentCount=0
        CommentCount=Comment(id)
        CommentIncrease=CommentCount-PreCommentCount
        print"更新:" +str(CommentIncrease)+"筆"
        

        LikeCount=0
        LikeCount=Like(id)
        LikeIncrease=LikeCount-PreLikeCount
        print"更新:" +str(LikeIncrease)+"筆"
        
        ShareIncrease=ShareCount-PreShareCount
        print"Shares更新:" +str(ShareIncrease)+"筆"
        
        insertSQL(id,PostNO)
        
        if (id==jd['data'][-1]['id']):
            url=jd['paging']['next']

            res=requests.get(url)
            jd = json.loads(res.text)
        
        PostNO+=1

  
    
        
        #停止點
        if (id=='15925638948_10154509502158949'):
            end=1
            
            break
            
    
conn.commit()
print "end"
cursor.close()
conn.close()   

Crawler_Time:11
11
評論載入成功:11筆 0.269999980927秒
評論輸入完畢:11筆 0.230999946594秒
更新:11筆
讚數載入成功:233個 0.31200003624秒
讚數輸入成功:233個 0.000999927520752秒
更新:233筆
Shares更新:31筆
評論載入成功:36筆 0.348000049591秒
評論輸入完畢:36筆 0.744000196457秒
更新:36筆
讚數載入成功:672個 0.536999940872秒
讚數輸入成功:672個 0.00100016593933秒
更新:672筆
Shares更新:147筆
end


In [ ]:
cursor.execute("select * from FBPostData   order by PostNO asc")
rows = cursor.fetchall()
print rows[0][0]


In [ ]:
cursor.close()
conn.close()  

In [80]:
A=2

b()

print A


4


In [78]:
def aa():
    global A
    A+=2
def b():
    
    aa()
    